
## Task & Quality Metrics


**Task & Quality metrics** measure **how good the model’s output is for the user’s task**.

They answer:

> **Did the model actually solve the problem correctly and well?**

These metrics are:

* Model-agnostic
* Task-dependent
* Used in **product evaluation, regression testing, and deployment gating**

---

### Core Task & Quality Metrics

| Metric           | What It Measures                |
| ---------------- | ------------------------------- |
| **Correctness**  | Factual accuracy                |
| **Relevance**    | Alignment with the question     |
| **Completeness** | Coverage of all required points |
| **Coherence**    | Logical flow                    |
| **Fluency**      | Language quality                |
| **Consistency**  | Stability across runs           |
| **Specificity**  | Avoids generic answers          |
| **Conciseness**  | Avoids unnecessary verbosity    |

---

### Why These Metrics Matter

A response can be:

* Fluent ❌
* Confident ❌
* Well-structured ❌
* **Wrong** ❌

Task & quality metrics ensure outputs are **useful, reliable, and trustworthy**.

---

### Demonstration Setup

#### Example Dataset



In [1]:
samples = [
    {
        "question": "What is RAG?",
        "ground_truth": "RAG combines document retrieval with language model generation."
    },
    {
        "question": "What is token streaming?",
        "ground_truth": "Token streaming sends output tokens incrementally."
    }
]




---

#### Example Model Pipeline



In [2]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import ChatPromptTemplate

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("Answer clearly: {question}")
chain = prompt | llm




---

#### Generate Predictions



In [3]:
results = []
for s in samples:
    out = chain.invoke({"question": s["question"]})
    results.append({**s, "prediction": out.content})




---

### Evaluate Task & Quality Metrics

#### Correctness



In [5]:
from langchain_classic.evaluation import load_evaluator

correctness_eval = load_evaluator("labeled_criteria", criteria="correctness")

for r in results:
    score = correctness_eval.evaluate_strings(
        input=r["question"],
        prediction=r["prediction"],
        reference=r["ground_truth"]
    )
    print("Correctness:", score["score"])


Correctness: 0
Correctness: 1




---

#### Relevance



In [6]:
relevance_eval = load_evaluator("criteria", criteria="relevance")

for r in results:
    score = relevance_eval.evaluate_strings(
        input=r["question"],
        prediction=r["prediction"]
    )
    print("Relevance:", score["score"])


Relevance: 0
Relevance: 0




---

#### Completeness



In [10]:
completeness_eval = load_evaluator(
    "labeled_criteria", 
    criteria={
        "completeness": "Does the submission cover all key points from the reference answer?"
    }
)

for r in results:
    score = completeness_eval.evaluate_strings(
        input=r["question"],
        prediction=r["prediction"],
        reference=r["ground_truth"]
    )
    print("Completeness:", score["score"])

Completeness: 0
Completeness: 1




---

#### Coherence & Fluency



In [12]:
coherence_eval = load_evaluator("criteria", criteria="coherence")
fluency_eval   = load_evaluator("criteria", criteria={
    "fluency": "Is the submission well-written, grammatically correct, and easy to read?"
})

for r in results:
    print("Coherence:", coherence_eval.evaluate_strings(
        input=r["question"], prediction=r["prediction"])["score"])
    
    print("Fluency:", fluency_eval.evaluate_strings(
        input=r["question"], prediction=r["prediction"])["score"])


Coherence: 1
Fluency: 1
Coherence: 1
Fluency: 1




---

#### Consistency (Multi-run)



In [13]:
answers = [chain.invoke({"question": "What is RAG?"}).content for _ in range(3)]
print("Consistency variation:", len(set(answers)))


Consistency variation: 3



Lower variation → higher consistency.

---

#### Specificity & Conciseness



In [15]:
specificity_eval = load_evaluator("criteria", criteria={
	"specificity": "Does the submission provide specific details rather than generic or vague statements?"
})
conciseness_eval = load_evaluator("criteria", criteria="conciseness")




---

### Aggregating Quality Score

```python
final_quality = (
    0.3 * correctness +
    0.25 * relevance +
    0.2 * completeness +
    0.15 * coherence +
    0.1 * fluency
)
```

---

### How Companies Use This

* Pre-release validation
* Regression detection
* Prompt tuning
* Model comparison
* Automated gating in CI/CD

---

### Mental Model

```
Task & Quality Metrics =
Does it work + Is it useful + Is it readable + Is it reliable
```

---

### Key Takeaways

* These metrics evaluate **actual output quality**
* They are independent of the model internals
* They form the backbone of LLM testing & deployment
* Must be automated & tracked continuously
